In [2]:
!pip install git+https://github.com/edtechre/pybroker.git

  Cloning https://github.com/edtechre/pybroker.git to /tmp/pip-req-build-sqffjx0r
  Running command git clone --filter=blob:none --quiet https://github.com/edtechre/pybroker.git /tmp/pip-req-build-sqffjx0r
  Resolved https://github.com/edtechre/pybroker.git to commit d40b5b02993b64608b4ce1a6ba810976c1410665
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.2 MB/s eta 0:00:00
  Created wheel for lib-pybroker: filename=lib

In [3]:
!pip install akshare numpy pandas

In [6]:
import pybroker
print(pybroker.__version__)  # 若无版本号，说明安装可能有问题

1.2.10


In [7]:
!python --version

Python 3.11.12


In [21]:
from pybroker import Strategy, StrategyConfig
from pybroker.ext.data import AKShare
import pandas as pd

# 策略逻辑：收盘价上穿/下穿5日均线
def handle_data(ctx):
    symbol = ctx.symbol
    close = ctx.close

    if len(close) < 6:
        return

    # 使用 Pandas 计算 5 日均线
    ma5_series = pd.Series(close).rolling(window=5).mean()
    ma5 = ma5_series.values

    # 获取持仓情况（注意 pos_type 应为 "long"）
    position = ctx.pos(symbol, "long")
    has_position = position and position.shares > 0

    # 上穿5日均线，买入
    if close[-1] > ma5[-1] and close[-2] <= ma5[-2]:
        if not has_position:
            ctx.buy_shares = 100

    # 下穿5日均线，卖出
    elif close[-1] < ma5[-1] and close[-2] >= ma5[-2]:
        if has_position:
            ctx.sell_all_shares()

# 初始化数据源
data_source = AKShare()

# 创建策略对象
strategy = Strategy(
    data_source,
    start_date="2023-01-01",
    end_date="2023-12-31",
    config=StrategyConfig(initial_cash=20000)
)

# 添加策略逻辑
strategy.add_execution(handle_data, symbols=["601318.SH"])  # 中国平安（上交所代码）

# 执行回测
result = strategy.backtest()

# 输出结果
print("总收益:", result.)
print("交易记录:\n", result.trades)


Backtesting: 2023-01-01 00:00:00 to 2023-12-31 00:00:00

Loading bar data...
Loaded bar data: 0:00:01 

Test split: 2023-01-03 00:00:00 to 2023-12-29 00:00:00


100% (242 of 242) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:08


AttributeError: 'TestResult' object has no attribute 'total_pnl'